In [1]:
import os
import math
import torch
import random
import pickle
import pandas
import calendar
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
os.environ['KMP_DUPLICATE_LIB_OK']='True'
torch.set_default_tensor_type(torch.FloatTensor)
import config

# Read and process data

In [2]:
X = torch.load('./data/sigmoid_RT_ratio.data')
X.shape

torch.Size([9999, 8])

In [3]:
X_max = torch.max(X, dim=0)[0]
X_min = torch.min(X, dim=0)[0]

In [4]:
X_max

tensor([7.4995e+05, 7.9995e+04, 6.0000e-04, 2.0000e-04, 8.0000e-04, 8.0000e-05,
        7.4875e+00, 1.9975e+01])

In [5]:
X_min

tensor([3.5005e+05, 4.0002e+04, 8.0000e-05, 8.0000e-05, 5.0000e-04, 4.0000e-05,
        4.0816e-01, 6.2500e+00])

In [7]:
Xn = config.Normalization(X, X_min, X_max)
Xn

tensor([[0.7501, 0.2500, 0.2500,  ..., 0.7500, 0.2120, 0.2992],
        [0.2499, 0.7501, 0.7500,  ..., 0.2500, 0.3329, 0.3825],
        [0.3750, 0.3750, 0.6250,  ..., 0.1250, 0.2516, 0.5355],
        ...,
        [0.5671, 0.4215, 0.4827,  ..., 0.6250, 0.2007, 0.3483],
        [0.0670, 0.9216, 0.9827,  ..., 0.1250, 0.6323, 0.4627],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.2854, 0.3339]])

In [9]:
Xn.max(0), Xn.min(0)

(torch.return_types.max(
 values=tensor([1., 1., 1., 1., 1., 1., 1., 1.]),
 indices=tensor([5459, 4367, 1732,  217, 1010,  101, 8830, 7476])),
 torch.return_types.min(
 values=tensor([0., 0., 0., 0., 0., 0., 0., 0.]),
 indices=tensor([8189, 8736, 1443,  136,  743,   74, 9826, 7679])))

In [10]:
Y = torch.load('./data/sigmoid_param.data')
Y.shape

torch.Size([9999, 4])

In [11]:
Y_max = torch.max(Y, dim=0)[0]
Y_min = torch.min(Y, dim=0)[0]

In [12]:
Y_min

tensor([ 0.0178,  0.8972, -0.1441,  8.9818], dtype=torch.float64)

In [13]:
Y_max

tensor([1.0265e-01, 9.8168e-01, 4.8684e-01, 4.1893e+02], dtype=torch.float64)

In [14]:
Yn = config.Normalization(Y, Y_min, Y_max)

In [15]:
Yn.max(0), Yn.min(0)

(torch.return_types.max(
 values=tensor([1., 1., 1., 1.], dtype=torch.float64),
 indices=tensor([ 716, 8989, 2493, 5775])),
 torch.return_types.min(
 values=tensor([0., 0., 0., 0.], dtype=torch.float64),
 indices=tensor([8989,  716,  136, 9981])))

# Make dataset

In [16]:
E_train = int(X.shape[0]*0.7)
E_valid = int(X.shape[0]*0.2)
E_test  = int(X.shape[0] - E_train - E_valid)

config.SetSeed()
index = torch.randperm(X.shape[0])
index_learn = index[:E_train+E_valid]
index_train = index[:E_train]
index_valid = index[E_train:E_train+E_valid]
index_test  = index[-E_test:]

X_learn, Y_learn = X[index_learn,:], Y[index_learn,:]
X_train, Y_train = X[index_train,:], Y[index_train,:]
X_valid, Y_valid = X[index_valid,:], Y[index_valid,:]
X_test,  Y_test  = X[index_test,:] , Y[index_test,:]

Xn_learn, Yn_learn = Xn[index_learn,:], Yn[index_learn,:]
Xn_train, Yn_train = Xn[index_train,:], Yn[index_train,:]
Xn_valid, Yn_valid = Xn[index_valid,:], Yn[index_valid,:]
Xn_test,  Yn_test  = Xn[index_test,:] , Yn[index_test,:]

In [17]:
a = {'Xn': Xn, 'Yn': Yn, 
     'X': X,   'Y': Y, 
     
     'Xn_learn': Xn_learn, 'Yn_learn': Yn_learn, 
     'Xn_train': Xn_train, 'Yn_train': Yn_train, 
     'Xn_valid': Xn_valid, 'Yn_valid': Yn_valid, 
     'Xn_test' : Xn_test,  'Yn_test' : Yn_test,
     
     'X_learn': X_learn, 'Y_learn': Y_learn, 
     'X_train': X_train, 'Y_train': Y_train, 
     'X_valid': X_valid, 'Y_valid': Y_valid, 
     'X_test' : X_test,  'Y_test' : Y_test,
     
     'X_max':X_max, 'X_min':X_min,
     'Y_max':Y_max, 'Y_min':Y_min,}

torch.save(a, './data/sigmoid.ds')